In [1]:
import sys, csv
from modules.wordTokenizer import WordTokenizer
from modules.LSTMNetwork import LSTMNetwork
from modules.baseline import Baseline
import numpy as np
import pandas as pd

Using TensorFlow backend.
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type

In [2]:
MAX_TEXTS = 1000000

In [13]:
#Read product file
texts, tags, brands = [], [], []
with open("data/train_products.csv", 'r') as f:
    reader = csv.DictReader(f, fieldnames=["title","brand","tags"])
    count = 0
    for row in reader:
        #print(row)
        count += 1
        text, tag_set = row['title'], row['tags'].split(' ')[:-1]
        texts.append(text)
        tags.append(tag_set)
        brands.append(row['brand'])
        if count >= MAX_TEXTS:
            break

In [14]:
#Create training examples
numSentences = len(texts)
np.random.seed(0)
training_examples = np.random.binomial(1, 0.7, numSentences)

In [15]:
#Separate data in training and test
trainData = []
testData  = []

nerLabels_train =[]
nerLabels_test = []


for example in range(numSentences):
    if training_examples[example] == 1:
        trainData.append(texts[example])
        nerLabels_train.append(tags[example])
    else:
        testData.append(texts[example])
        nerLabels_test.append(tags[example])

In [7]:
len(trainData)

43626

In [16]:
#create subset of train data
numTrainSentences = 40000

trainData_tiny       = trainData[0:numTrainSentences]
nerLabels_train_tiny = nerLabels_train[0:numTrainSentences]

In [13]:
len(trainData_tiny)

20000

In [17]:
#Initiate network
network = LSTMNetwork()

In [18]:
#Create tokenized sentences
wordTokenizer = WordTokenizer(30,'models/word_tokenizer')
wordTokenizer.load()
data = wordTokenizer.tokenize(trainData_tiny)
testData = wordTokenizer.tokenize(testData)

In [19]:
#Generate pad labels for network
labels = network.get_labels(nerLabels_train_tiny,wordTokenizer)
testLabels = network.get_labels(nerLabels_test,wordTokenizer)

Getting labels...
Getting labels...


In [20]:
network.compile(wordTokenizer)
network.train(data, labels, epochs=8, validation_split=0.2, batch_size=32)

Loading spanish embedding...
Found 855381 word vectors.
Creating embedding layer...
Creating network...
{'O': 1, 'B-B': 2, 'I-B': 3}
Compiling network...
Training...
(40000, 30) (40000, 30, 3)
Train on 32000 samples, validate on 8000 samples
Epoch 1/8
32000/32000 [==============================] - 130s 4ms/step - loss: 0.2998 - acc: 0.8910 - val_loss: 0.1657 - val_acc: 0.9441

Epoch 00001: saving model to models/lstm.h5
Epoch 2/8
32000/32000 [==============================] - 126s 4ms/step - loss: 0.1771 - acc: 0.9381 - val_loss: 0.1342 - val_acc: 0.9523

Epoch 00002: saving model to models/lstm.h5
Epoch 3/8
32000/32000 [==============================] - 124s 4ms/step - loss: 0.1492 - acc: 0.9471 - val_loss: 0.1172 - val_acc: 0.9590

Epoch 00003: saving model to models/lstm.h5
Epoch 4/8
32000/32000 [==============================] - 125s 4ms/step - loss: 0.1331 - acc: 0.9525 - val_loss: 0.1081 - val_acc: 0.9618

Epoch 00004: saving model to models/lstm.h5
Epoch 5/8
32000/32000 [=======

In [22]:
network.evaluate(testData,testLabels)

Evaluating...
18558/18558 [==============================] - 3s 180us/step
[0 0 0 ... 0 0 0]
              precision    recall  f1-score   support

           O   0.996163  0.993797  0.994978    532625
         B-B   0.839596  0.895663  0.866724     18584
         I-B   0.939514  0.943591  0.941548      5531

    accuracy                       0.990022    556740
   macro avg   0.925091  0.944350  0.934417    556740
weighted avg   0.990374  0.990022  0.990166    556740



In [ ]:
x = [500, 16000, 32000]
y = []